# Getting Started
In this notebook you will learn how to run your first simulation with fuse.


## Imports

In [ ]:
import fuse
import matplotlib.pyplot as plt
import numpy as np
from straxen import URLConfig

## Setting up the simulation context

Just like for our XENON analysis software, fuse simulations are run in a so called context. The context defines which plugins are used and how they are configured. For this tutorial we will run a so called "full chain" simulation. This means that our input will be a root file from a Geant4 simulation and our output will be raw_records. 

At the moment the simulation contexts are defined in `fuse.context`. When fuse will be made public, we will move the XENONnT specific context to cutax. To build our context we need to load a configuration file first. This file contains all needed information to configure the plugins. To load the config file we can use the URLConfig class of straxen. 

In [ ]:
url_string = 'simple_load://resource://format://fax_config_nt_sr0_v4.json?&fmt=json'
config = URLConfig.evaluate_dry(url_string)

Now that we have our config we can define our context. There can be some warings related to numba, urlconfigs and the simulation context. These can be ignored and the warnings related to fuse will be fixed in the future.

After building the context we need to tell fuse which file to use as input. This is done via the config options `path` and `file_name`. To set these options we can use the `set_config` method of the context. Any other config option can be changed in the same way.

In [ ]:
st = fuse.context.full_chain_context(out_dir = "./fuse_data",
                                     config = config)

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "entry_stop": 10,
               })

## Running the simulation
Now we have everything ready for the first simulation. We will tell fuse to simulate raw_records via the command `st.make(run_number, 'raw_records')`. The first argument is the run number, which is used to identify the data afterwards. The second argument is the data type we want to simulate and save. We can in principle use any run_number we like.

In [ ]:
run_number = "00000"
st.make(run_number,"raw_records")

With default settings fuse will only save `microphysics_summary`and `raw_records`. The first one is the output of the microphysics simulation corresponding to the previously used epix software. The second one is the output of the full chain simulation (microphysics, detector physics and PMT + DAQ simulation) corresponding to the output of WFSin. 

## Loading the data
Now that our simulation finished we can load the simulaiton result. This is also done via the methods strax provides. You can either use `st.get_array(run_number, "target)` to get the data in numpy format or use `st.get_df(run_number, "target")` to get a pandas dataframe. Please note that not all data types can be loaded as dataframe. First lets look at the output of the microphysics simulation. 

In [ ]:
microphysics_summary = st.get_df(run_number, "microphysics_summary")

In [ ]:
microphysics_summary.head()

Lets look at the `raw_records` next. They need to be loaded as numpy array. 

In [ ]:
raw_records = st.get_array(run_number, "raw_records")

We will end this notebook with a simple plot of one `raw_record`. If you want to learn more about fuse, please have a look at the other notebooks.

In [ ]:
record_to_plot = raw_records[123]

time = np.linspace(record_to_plot["time"], record_to_plot["time"] + record_to_plot["length"]*record_to_plot["dt"], record_to_plot["length"])

plt.plot(time, record_to_plot["data"][0:record_to_plot["length"]])

plt.xlabel("Time (ns)")
plt.ylabel("Amplitude (ADC)")
plt.show()